# scrapping 대상 사이트
1. 블라인드 : https://www.teamblind.com/kr/search/%EB%94%94%EC%8A%A4%ED%81%AC
2. 하이닥 : https://www.hidoc.co.kr/integratesearch/searchhealthqnalist?query=%EB%94%94%EC%8A%A4%ED%81%AC&page=1
3. 클리앙 : https://www.clien.net/service/search?q=%EB%94%94%EC%8A%A4%ED%81%AC&sort=recency&p=10&boardCd=&isBoard=false

In [25]:
from selenium import webdriver
import time

In [24]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../chromedriver.exe')

## 1. 블라인드 사이트

In [25]:
browser.get('https://www.teamblind.com/kr/search/%EB%94%94%EC%8A%A4%ED%81%AC')

In [26]:
browser.implicitly_wait(10)

### 게시글 클릭
- #wrap > section > div > div > div.article-list > div

In [47]:
click_path = '#wrap > section > div > div > div.article-list > div'
browser.find_element_by_css_selector(click_path).click()

### 게시글 정보
- title tag(제목) : div.article-view-head > h2
- content tag(내용) : .contents-txt
- review tag(댓글) : .wrap-comment.comment_area
- reply(답글) tag : .wrap-reply

In [48]:
# 글 1개의 제목, 내용
title_name = browser.find_element_by_css_selector('div.article-view-head > h2').text
content = browser.find_element_by_css_selector('.contents-txt').text

[title_name, content]

['허리 디스크 여자 다이어트',
 '안녕하세요!\n운동에 대해 잘 몰라서 여쭤보러고 글 남깁니다.\n\n저는 현재 매우 비만 상태고\n164/75(추정)\n운동도 거의해본 적 없는 상태라 다이어트를 하려고 했는데 허리를 다쳤어요ㅠㅠ\n\n식단으로만 빼기에는 체질 개선도 그렇고 살쳐짐도 있을 것 같아 운동을 병행하기는 해야할 거 같은데\n허리 디스크가 있는 경우 어떤 운동을 해야 하나요?ㅠ\n\n디스크는 확진 판정을 받은 건 아니고 (mri찍은 게 아니어서..)\n허리를 삐끗해서 정형외과 갔고\n현재 약물 및 물리치료 1번 받고 한의원 갔다가 효과가 없어서 정형외과에서 허리 주사를 한 번 받았습니다.\n\n통증은 거의 사라졌지만 자세 신경쓰고 쉬어서 그런거 같거 움직이면 약한 통증은 있어서 주사 한 번 더 맞을 거 같아요..\n\n이런 경우 어떤 운동을 할 수 있을까요?\n\n헬스장 런닝머신이나 스텝밀(천국의 계단)은 해도 될까요..?\n아니면 필라테스 등으로 자세를 교정하는 게 좋을까요?\n\n조언 부탁드리겠습니다! ㅠㅠ']

In [31]:
# 리뷰
reviews_bundle = browser.find_elements_by_css_selector('.wrap-comment.comment_area')
len(reviews_bundle)
for review in reviews_bundle :
    try:
        review_writer = review.find_element_by_css_selector('p.name').text
        review_text = review.find_element_by_css_selector('.cmt-txt').text
        print('writer: {}'.format(review_writer))
        print('review: {}'.format(review_text))
    except :
        print(str())
print('Done')

Done


In [50]:
# 첫 페이지에서 보여지는 리스트(50개)
# 스크롤 하면 20개씩 증가(50->70->90...)
reviews_standard_count = browser.find_elements_by_css_selector('.article-list-pre')
len(reviews_standard_count)

50

In [36]:
import pandas as pd
import pymongo as mg
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['small_team_project']
collection = database['teamblind_site']

teamblind_columns_name = ['writer', 'date', 'review']
basket_list= []
from selenium.webdriver.common.keys import Keys
# 화면 끝까지 이동
screen_height = browser.execute_script('return window.innerHeight')
document_height = browser.execute_script('return document.body.scrollHeight')
screen_height, document_height
i=4

# 스크롤
while True:
    # 글 리스트 가져오기
    for clicker in range(2, 3598):
        time.sleep(1)
        click_path = 'div:nth-child({}) > div.tit > h3'.format(clicker)
        time.sleep(1)
        browser.find_element_by_css_selector(click_path).click()
        # 작성자, 작성일, 내용 가져오기
        reviews_bundle = browser.find_elements_by_css_selector('.wrap-comment.comment_area')
        for review in reviews_bundle :
            try:
                review_writer = review.find_element_by_css_selector('p.name').text
            except :
                print(str())
            try:
                review_date = review.find_element_by_css_selector('.date').text
            except :
                print(str())
            try:
                review_text = review.find_element_by_css_selector('.cmt-txt').text
            except :
                print(str())
        
        total_list = [review_writer, review_date, review_text]
        basket_list.append(total_list)

        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=basket_list, columns=teamblind_columns_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)

        browser.back() # 리스트로 돌아가기
        
        if clicker == i  :
            browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
            i=i+4
        
if current_scroll_position >= document_height :
    reviews_standard_count = browser.find_elements_by_css_selector('.article-list-pre')
    print(len(reviews_standard_count))
    break
        
print('Done')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div:nth-child(272) > div.tit > h3"}
  (Session info: chrome=116.0.5845.188)


In [ ]:
# browser.back() # 리스트로 돌아가기

In [ ]:
# browser.quit()